# 8 머신러닝 기초(지도학습)

***

In [ ]:
# 데이터 가공, 조작, 분석 라이브러리
import numpy as np
import numpy.random as random
import scipy as sp
from pandas import Series, DataFrame
import pandas as pd

# 시각화 라이브러리
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
%matplotlib inline

# 머신러닝 라이브러리
import sklearn

# 소수점 3자리까지 표시
%precision 3

In [ ]:
# 임포트
import requests, zipfile
import io

# 자동차 가격 데이터 가져오기
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/autos/imports-85.data'
res = requests.get(url).content

# 데이터를 DataFrame 객체로 읽어 들이기
auto = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# 데이터 컬럼에 레이블 설정
auto.columns =['symboling','normalized-losses','make','fuel-type' ,'aspiration','num-of-doors',
                            'body-style','drive-wheels','engine-location','wheel-base','length','width','height',
                            'curb-weight','engine-type','num-of-cylinders','engine-size','fuel-system','bore',
                            'stroke','compression-ratio','horsepower','peak-rpm','city-mpg','highway-mpg','price']

In [ ]:
print('자동차 데이터 형태:{}'.format(auto.shape))

In [ ]:
auto.head()

In [ ]:
# 각 컬럼 데이터에 포함된 ? 개수 확인
auto = auto[['price','horsepower','width','height']]
auto.isin(['?']).sum()

In [ ]:
# ?를 NaN으로 전환하고 NaN이 있는 행 데이터 제거
auto = auto.replace('?', np.nan).dropna()
print('자동차 데이터 형태:{}'.format(auto.shape))

In [ ]:
print('데이터 형 확인(형 변환 전)\n{}\n'.format(auto.dtypes))

In [ ]:
auto = auto.assign(price=pd.to_numeric(auto.price))
auto = auto.assign(horsepower=pd.to_numeric(auto.horsepower))
print('데이터 형 확인(형 변환 후）\n{}'.format(auto.dtypes))

In [ ]:
auto.corr()

In [ ]:
# 데이터 분할(훈련 데이터와 테스트 데이터)을 위한 임포트
from sklearn.model_selection import train_test_split

# 다중회귀 모델 구축을 위한 임포트
from sklearn.linear_model import LinearRegression

# 목표변수로 price를 지정, 설명변수로는 price 외의 다른 변수를 지정
X = auto.drop('price', axis=1)
y = auto['price']

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 다중회귀 클래스 초기화와 학습
model = LinearRegression()
model.fit(X_train,y_train)

# 결정계수 출력
print('결정계수(train):{:.3f}'.format(model.score(X_train,y_train)))
print('결정계수(test):{:.3f}'.format(model.score(X_test,y_test)))
 
# 회귀계수와 절편 출력
print('\n회귀계수\n{}'.format(pd.Series(model.coef_, index=X.columns)))
print('절편: {:.3f}'.format(model.intercept_))

In [ ]:
# 데이터 수집
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data'
res = requests.get(url).content

# 수집한 데이터를 DataFrame 객체로 읽어 들이기
adult = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# 데이터 컬럼에 레이블 설정
adult.columns =['age','workclass','fnlwgt','education','education-num','marital-status',
                             'occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week',
                             'native-country','flg-50K']


# 데이터 형식과 결측값 개수 출력
print('데이터 형식:{}'.format(adult.shape))
print('결측값 수:{}'.format(adult.isnull().sum().sum()))

# 데이터의 처음 5행 출력
adult.head()

In [ ]:
adult.groupby('flg-50K').size()

In [ ]:
# flg-50K 컬럼 값이 >50K이면 1, 아니면 0인 fin_flg 변수 추가 생성
adult['fin_flg'] = adult['flg-50K'].map(lambda x: 1 if x ==' >50K' else 0)
adult.groupby('fin_flg').size()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 설명변수와 목표변수 설정
X = adult[['age','fnlwgt','education-num','capital-gain','capital-loss']]
y = adult['fin_flg']

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 로지스틱회귀 클래스 초기화와 학습
model = LogisticRegression()
model.fit(X_train,y_train)

print('정확도(train):{:.3f}'.format(model.score(X_train, y_train)))
print('정확도(test):{:.3f}'.format(model.score(X_test, y_test)))

In [ ]:
model.coef_

In [ ]:
np.exp(model.coef_)

In [ ]:
# 표준화를 위한 클래스 임포트
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# X와 y 설정
X = adult[['age','fnlwgt','education-num','capital-gain','capital-loss']]
y = adult['fin_flg']

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 표준화 처리
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# 로지스틱회귀 클래스 초기화와 학습
model = LogisticRegression()
model.fit(X_train_std,y_train)

# 정확도 출력
print('정확도(train):{:.3f}'.format(model.score(X_train_std, y_train)))
print('정확도(test):{:.3f}'.format(model.score(X_test_std, y_test)))

In [ ]:
auto.head()

In [ ]:
# 릿지회귀용 클래스
from sklearn.linear_model import Ridge
from sklearn.model_selection import train_test_split

# 훈련 데이터와 테스트 데이터로 분할
X = auto.drop('price', axis=1)
y = auto['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5, random_state=0)

# 모델 구축과 평가
linear = LinearRegression()
ridge = Ridge(random_state=0)

for model in [linear, ridge]:
    model.fit(X_train,y_train)
    print('{}(train):{:.6f}'.format(model.__class__.__name__ , model.score(X_train,y_train)))
    print('{}(test):{:.6f}'.format(model.__class__.__name__ , model.score(X_test,y_test)))

In [ ]:
# 데이터 수집
url = 'http://archive.ics.uci.edu/ml/machine-learning-databases/mushroom/agaricus-lepiota.data'
res = requests.get(url).content

# 수집한 데이터를 DataFrame 객체로 읽어 들이기
mushroom = pd.read_csv(io.StringIO(res.decode('utf-8')), header=None)

# 데이터 컬럼에 레이블 설정
mushroom.columns =['classes','cap_shape','cap_surface','cap_color','odor','bruises',
                             'gill_attachment','gill_spacing','gill_size','gill_color','stalk_shape',
                             'stalk_root','stalk_surface_above_ring','stalk_surface_below_ring',
                             'stalk_color_above_ring','stalk_color_below_ring','veil_type','veil_color',
                             'ring_number','ring_type','spore_print_color','population','habitat']

# 처음 5행 출력
mushroom.head()

In [ ]:
print('데이터 형태:{}'.format(mushroom.shape))
print('결측값 수:{}'.format(mushroom.isnull().sum().sum()))

In [ ]:
mushroom_dummy = pd.get_dummies(mushroom[['gill_color','gill_attachment','odor','cap_color']])
mushroom_dummy.head()

In [ ]:
# 목표변수도 수치화(0/1)함
mushroom_dummy['flg'] = mushroom['classes'].map(lambda x: 1 if x =='p' else 0)

In [ ]:
mushroom_dummy.groupby(['cap_color_c', 'flg'])['flg'].count().unstack()

In [ ]:
mushroom_dummy.groupby(['gill_color_b', 'flg'])['flg'].count().unstack()

In [ ]:
- (0.5 * np.log2(0.5) + 0.5 * np.log2(0.5))

In [ ]:
- (0.001 * np.log2(0.001) + 0.999 * np.log2(0.999))

In [ ]:
def calc_entropy(p):
    return - (p * np.log2(p) + (1 - p) *  np.log2(1 - p) )

In [ ]:
# p값을 0.001에서 0.999까지 0.01 단위로 이동
p = np.arange(0.001, 0.999, 0.01)

# 그래프 출력
plt.plot(p, calc_entropy(p)) 
plt.xlabel('prob')
plt.ylabel('entropy')
plt.grid(True)

In [ ]:
mushroom_dummy.groupby('flg')['flg'].count()

In [ ]:
entropy_init = - (0.518 * np.log2(0.518) + 0.482 * np.log2(0.482))
print('독버섯 데이터 엔트로피 초깃값: {:.3f}'.format(entropy_init))

In [ ]:
mushroom_dummy.groupby(['cap_color_c', 'flg'])['flg'].count().unstack()

In [ ]:
# cap_color가 c가 아닐 경우 엔트로피
p1 = 4176 / (4176 + 3904)
p2 = 1 - p1
entropy_c0 = -(p1*np.log2(p1)+p2*np.log2(p2))
print('entropy_c0: {:.3f}'.format(entropy_c0))

In [ ]:
# cap_color가 c일 경우 엔트로피
p1 = 32/(32+12)
p2 = 1 - p1
entropy_c1 = -(p1*np.log2(p1)+p2*np.log2(p2))
print('entropy_c1: {:.3f}'.format(entropy_c1))

In [ ]:
entropy_after = (4176+3904)/8124*entropy_c0 + (32+12)/8124*entropy_c1
print('데이터 분할 후 평균 엔트로피: {:.3f}'.format(entropy_after))

In [ ]:
print('변수 cap_color 기준으로 분할했을 때의 정보이득: {:.3f}'.format(entropy_init - entropy_after))

In [ ]:
mushroom_dummy.groupby(['gill_color_b', 'flg'])['flg'].count().unstack()

In [ ]:
# gill_color가 b가 아닌 경우 엔트로피
p1 = 4208/(4208+2188)
p2 = 1 - p1
entropy_b0 = - (p1*np.log2(p1) + p2*np.log2(p2))

# gill_color가 b인 경우 엔트로피
p1 = 0/(0+1728)
p2 = 1 - p1
entropy_b1 = - (p2*np.log2(p2))

entropy_after = (4208+2188)/8124*entropy_b0 + (0+1728)/8124*entropy_b1
print('변수 gill_color 기준으로 분할했을 때의 정보이득: {:.3f}'.format(entropy_init - entropy_after))

In [ ]:
from sklearn.tree import  DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# 데이터 분할
X = mushroom_dummy.drop('flg', axis=1)
y = mushroom_dummy['flg']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

# 의사결정나무 클래스 초기화와 학습
model = DecisionTreeClassifier(criterion='entropy', max_depth=5, random_state=0)
model.fit(X_train,y_train)

print('정확도(train):{:.3f}'.format(model.score(X_train, y_train)))
print('정확도(test):{:.3f}'.format(model.score(X_test, y_test)))

In [ ]:
# 참고 프로그램
# pydotplus와 graphviz를 설치해야 합니다. (이 책에서는 설치 방법을 다루지 않습니다)
from sklearn import tree
import pydotplus
from sklearn.externals.six import StringIO
from IPython.display import Image

dot_data = StringIO()
tree.export_graphviz(model, out_file=dot_data)
graph = pydotplus.graph_from_dot_data(dot_data.getvalue())
Image(graph.create_png())  

In [ ]:
# 데이터와 모델을 구축하기 위한 라이브러리 임포트
from sklearn.datasets import load_breast_cancer
from sklearn.neighbors import  KNeighborsClassifier
from sklearn.model_selection import train_test_split

# 데이터세트 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터 분할
# stratify는 층화추출4
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify = cancer.target, random_state=0)

# 그래프 생성을 위한 리스트 생성
training_accuracy = []
test_accuracy =[]

# 학습
for n_neighbors in range(1,21):
    model = KNeighborsClassifier(n_neighbors=n_neighbors)
    model.fit(X_train,y_train)
    training_accuracy.append(model.score(X_train, y_train))
    test_accuracy.append(model.score(X_test, y_test))

# 그래프 출력
plt.plot(range(1,21), training_accuracy, label='Training')
plt.plot(range(1,21), test_accuracy, label='Test')
plt.ylabel('Accuracy')
plt.xlabel('n_neighbors')
plt.legend()

In [ ]:
student = pd.read_csv('student-mat.csv', sep=';')
X = student.loc[:, ['age','Medu','Fedu','traveltime','studytime'
                              ,'failures','famrel','freetime','goout','Dalc','Walc'
                              ,'absences','G1','G2']].values

In [ ]:
# SV 라이브러리
from sklearn.svm import LinearSVC

# 훈련 데이터와 테스트 데이터로 분할하는 라이브러리
from sklearn.model_selection import train_test_split

# 데이터 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify = cancer.target, random_state=0)

# 클래스 초기화와 학습
model = LinearSVC()
model.fit(X_train,y_train)

# 훈련 데이터와 테스트 데이터에서의 점수
print('정확도(train):{:.3f}'.format(model.score(X_train, y_train)))
print('정확도(test):{:.3f}'.format(model.score(X_test, y_test)))

In [ ]:
# 데이터 읽어 들이기
cancer = load_breast_cancer()

# 훈련 데이터와 테스트 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(
    cancer.data, cancer.target, stratify = cancer.target, random_state=0)

# 표준화
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

# 클래스 초기화와 학습
model = LinearSVC()
model.fit(X_train_std,y_train)

# 훈련 데이터와 테스트 데이터에서의 점수
print('정확도(train):{:.3f}'.format(model.score(X_train_std, y_train)))
print('정확도(test):{:.3f}'.format(model.score(X_test_std, y_test)))